In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.tree import RandomForest, RandomForestModel

from pyspark.mllib.util import MLUtils

%pylab inline

import numpy as np
from time import time

from DistributedBoosting import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pyspark import SparkContext

sc=SparkContext()

from numpy.random import rand
p=0.9
data=[]
for i in range(10):
    for j in range(10):
        if np.abs(i-4)<3 and np.abs(j-6)<3:
            y=2*(rand()<p)-1
        else:
            y=2*(rand()>p)-1
        print "%1.0f "%((1+y)/2),
        data.append(LabeledPoint(y,[i,j]))
    print

dataRDD=sc.parallelize(data,numSlices=2)
dataRDD.getNumPartitions()

1  0  0  0  0  0  0  0  0  0 
0  1  1  0  0  0  0  0  0  1 
0  0  0  0  1  1  1  1  1  0 
0  0  0  0  1  0  1  1  0  0 
0  0  0  0  1  1  1  1  0  0 
1  0  0  0  1  1  1  1  1  1 
0  0  0  0  1  0  1  1  1  0 
0  0  0  0  0  0  0  0  0  0 
0  0  0  0  0  0  0  0  1  0 
0  0  1  0  0  0  0  0  1  0 


2

In [3]:
# %load DistributedBoosting.py
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.tree import RandomForest, RandomForestModel

from pyspark.mllib.util import MLUtils

import numpy as np
from time import time
from string import split,strip

In [4]:
class Timer:
    """A simple service class to log run time and pretty-print it.
    """
    def __init__(self):
        self.T=[]
    def stamp(self,name):
        self.T.append((name,time()))
    def str(self):
        T=self.T
        return '\n'.join(['%6.2f : %s'%(T[i+1][1]-T[i][1],T[i+1][0]) for i in range(len(T)-1)])

In [5]:
###### Globals
global T,iteration,GR,proposals,Strong_Classifier, feature_no, partition_no, Splits_Table
global Strong_Classifier,global_best_splitter,PS

T=Timer()
feature_no=None                 # Tracks processing time
global_feature_no=None
partition_no=0
iteration=0                     # Boosting iteration
PS=[None]                       # RDD that hold state of boosting process for each partition.
proposals=[]                    # proposed splits for each feature
Strong_Classifier=[]            # Combined weak classifiers
#############################################################

In [6]:
##### Partition fundctions
def Prepare_partition_data_structure(A):

    rows=len(A[1])

    columns=np.empty([feature_no,rows])
    columns[:]=np.NaN
    print 'Prepare_partition_data_structure',feature_no,np.shape(columns)
    
    labels=np.empty(rows)
    labels[:]=np.NaN

    for j in range(rows):
        LP=A[1][j]
        labels[j]=LP.label
        for i in range(feature_no):
            columns[i,j]=LP.features[i]
    return {'index':A[0],\
            'labels':labels,\
            'weights':np.ones(len(labels)),\
            'feature_values':columns}

def Add_weak_learner_matrix(A):
    """ This procedure adds to each partition the matrix that will be 
        used to efficiently find the best weak classifier """

    try:
        feature_no
    except:
        feature_no=global_feature_no.value

    index=A['index']%feature_no
    SP=Splits_Table.value[index]

    Col=A['feature_values'][index,:]

    ### The matrix M is organized as follows: 
    # * There are as many rows as there are thresholds in SP (last one is inf)
    # * There are as many columns as there are examples in this partition.
    # For threshold i, the i'th rw of M is +1 if Col is smaller than the trehold SP[i] and -1 otherwise

    M=np.empty([len(SP),len(Col)])
    M[:]=np.NaN

    for i in range(len(SP)):
        M[i,:]=2*(Col<SP[i])-1

    A['M']=M # add M matrix to the data structure.
    return A


def Find_weak(A):
    """Find the best split for a single feature on a single partition"""

    try:
        feature_no
    except:
        feature_no=global_feature_no.value

    index=A['index']%feature_no
    SP=Splits_Table.value[index]

    M=A['M']
    weights=A['weights']
    weighted_Labels=weights*A['labels']
    SS=np.dot(M,weighted_Labels)/np.sum(weights)
    i_max=np.argmax(np.abs(SS))
    answer={'Feature_index':A['index']%feature_no,\
            'Threshold_index':i_max,\
            'Threshold':SP[i_max],\
            'Correlation':SS[i_max],\
            'SS':SS}
    return answer

# update weights. New splitter is shipped to partition as one of the referenced
# Variables

def update_weights(A):
    """Update the weights of the exammples belonging to this 
    partition according to the new splitter"""
    best_splitter=global_best_splitter
    F_index=best_splitter['Feature_index']
    Thr=best_splitter['Threshold']
    alpha=best_splitter['alpha']
    y_hat=2*(A['feature_values'][F_index,:]<Thr)-1
    y=A['labels']
    weights=A['weights']*exp(-alpha*y_hat*y)
    weights /= sum(weights)
    A['weights']=weights
    return A

def calc_scores(Strong_Classifier,Columns,Lbl):
    
    Scores=np.zeros(len(Lbl))

    for h in Strong_Classifier:
        index=h['Feature_index']
        Thr=h['Threshold']
        alpha=h['alpha']
        y_hat=2*(Columns[index,:]<Thr)-1
        Scores += alpha*y_hat*Lbl
    return Scores

In [8]:
if __name__ == '__main__':
    from os.path import exists
    if not exists('higgs'):
        print "creating directory higgs"
        get_ipython().system(u'mkdir higgs')
    get_ipython().magic(u'cd higgs')
    if not exists('HIGGS.csv'):
        if not exists('HIGGS.csv.gz'):
            print 'downloading HIGGS.csv.gz'
            get_ipython().system(u'curl -O http://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz')
        print 'decompressing HIGGS.csv.gz --- May take 5-10 minutes'
        get_ipython().system(u'gunzip -f HIGGS.csv.gz')
    get_ipython().system(u'ls -l')

    #copy file to HDFS - when runnnig on AWS cluster
    get_ipython().system(u'/root/ephemeral-hdfs/bin/hdfs dfs -cp file:///mnt/higgs/HIGGS.csv /HIGGS.csv')

def test_globals():
    return globals()

creating directory higgs
/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/src/pyspark_notebooks/TreesAndBoosting/higgs
downloading HIGGS.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  2 2685M    2 75.5M    0     0  6537k      0  0:07:00  0:00:11  0:06:49 6800k^C
decompressing HIGGS.csv.gz --- May take 5-10 minutes
gunzip: HIGGS.csv.gz: unexpected end of file
gunzip: HIGGS.csv.gz: uncompress failed
total 168472
-rw-r--r--  1 yoavfreund  staff  86257664 May 24 07:03 HIGGS.csv.gz
/bin/sh: /root/ephemeral-hdfs/bin/hdfs: No such file or directory


In [7]:
###### Head-Node functions
def init(sc,Data):
    """ Given an RDD with labeled Points, create the RDD of data structures used for boosting
    """

    global T,iteration,GR,proposals,Strong_Classifier, feature_no, partition_no, Splits_Table
    global Strong_Classifier,global_best_splitter

    T=Timer()
    T.stamp('Started')

    X=Data.first()
    feature_no=len(X.features)
#    print 'global_feature_no = sc.broadcast(feature_no)',feature_no
    partition_no=Data.getNumPartitions()
    if partition_no != feature_no:
        Data=Data.repartition(feature_no).cache()
    print 'number of features=',feature_no,'number of partitions=',Data.getNumPartitions()

    # Split data into training and test
    (trainingData,testData)=Data.randomSplit([0.7,0.3])
    print 'Sizes: Data1=%d, trainingData=%d, testData=%d'%      (Data.count(),trainingData.cache().count(),testData.cache().count())
    T.stamp('Split into train and test')
    # Glom each partition into a local array
    G=trainingData.glom()
    GTest=testData.glom()  
    T.stamp('glom')

    # Add an index to each partition to identify it.
    def f(splitIndex, iterator): yield splitIndex,iterator.next()
    GI=G.mapPartitionsWithIndex(f)
    GTI=GTest.mapPartitionsWithIndex(f)
    T.stamp('add partition index')

    return GI

In [10]:
GI=init(sc,dataRDD)

number of features= 2 number of partitions= 2
Sizes: Data1=100, trainingData=67, testData=33


In [11]:
def init2(GI):
    # Prepare the data structure for each partition.
    GR=GI.map(Prepare_partition_data_structure)
    print 'number of elements in GR=', GR.cache().count()
    T.stamp('Prepare_partition_data_structure')

    #compute the split points for each feature
    Splits=find_splits(GR)
    print 'Split points=',Splits
    T.stamp('Compute Split points')

    #broadcast split points
    global Splits_Table
    Splits_Table=sc.broadcast(Splits)
    T.stamp('Broadcast split points')

    # Create matrix for each partition to make finding the weak rules correlation a matter of taking a matrix product

    iteration=0
    global PS
    PS[0]=GR.map(Add_weak_learner_matrix)
    print 'number of partitions in PS=',PS[0].cache().count()
    T.stamp('Add_weak_learner_matrix')

    return PS

In [12]:
init2(GI)

number of elements in GR= 2


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 5.0 failed 1 times, most recent failure: Lost task 1.0 in stage 5.0 (TID 10, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/yoavfreund/spark-latest/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/yoavfreund/spark-latest/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/yoavfreund/spark-latest/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "DistributedBoosting.py", line 254, in find_split_points
    feature_no=global_feature_no.value
AttributeError: 'NoneType' object has no attribute 'value'

	at org.apache.spark.api.python.Pyth

In [ ]:
def boosting_iteration(k=1):
    """ perform k boosting iterations """
    for i in range(iteration,iteration+k):
        T.stamp('Start main loop %d'%i)

        prop=PS[i].map(Find_weak).collect()
        proposals.append(prop)
        corrs=[p['Correlation'] for p in prop]
        best_splitter_index=np.argmax(np.abs(corrs))
        best_splitter = prop[best_splitter_index]
        Strong_Classifier.append(best_splitter)
        global global_Strong_Classifier
        global_Strong_Classifier=sc.broadcast(Strong_Classifier)
        T.stamp('found best splitter %d'%i)

        corr=best_splitter['Correlation']
        best_splitter['alpha']=0.5*np.log((1+corr)/(1-corr))
        global global_best_splitter
        global_best_splitter = sc.broadcast(best_splitter)
        PS.append(PS[i].map(update_weights))
        T.stamp('Updated Weights %d'%i)
    iteration+=k

In [ ]:
def find_splits(GR,number_of_bins=10,debug=False):
    """Compute the split points for each feature to create number_of_bins bins"""
    def find_split_points(A):

        try:
            feature_no
        except:
            feature_no=global_feature_no.value

        j=A['index'] % feature_no
        S=np.sort(A['feature_values'][j,:       ])
        L=len(S) 
        step=L/number_of_bins+2*number_of_bins
        return (j,S[range(0,L,step)])

    global partition_no
    Splits=GR.map(find_split_points).collect()
    max_no=np.array([np.finfo(float).max])

    # Average the split points across the partitions corresponding to the same feature.
    Splits1=[]
    for i in range(feature_no):
        S=Splits[i][1]
        if debug:
            print 'no. ',i,' = ',Splits[i]
        n=1  # number of copies (for averaging)
        j=i+feature_no
        while j<partition_no:
            if debug:
                print 'j=',j
            S+=Splits[j][1]
            if debug:
                print 'no. ',j,' = ',Splits[j]
            n+=1.0
            j+=feature_no
        Splits1.append(np.concatenate([S/n,max_no]))
        if debug:
            print n
            print Splits1[i]
            print '='*60

    return Splits1